In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, StringType, IntegerType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load data from Unity Catalog tables
try:
    hospital_stats_df = spark.table("catalog.db.hospital_stats_north_america")
    sales_assignments_df = spark.table("catalog.db.hospital_sales_assignments")
    employment_details_df = spark.table("catalog.db.sales_associates_employment_details")
    compensation_guidelines_df = spark.table("catalog.db.compensation_guidelines")
    logistics_channels_df = spark.table("catalog.db.logistics_channels")
    growth_opportunities_df = spark.table("catalog.db.growth_opportunities")
    company_goals_df = spark.table("catalog.db.company_goals")
    historical_sales_df = spark.table("catalog.db.historical_sales")
    third_party_sales_trends_df = spark.table("catalog.db.third_party_sales_trends")
except Exception as e:
    logger.error(f"Error loading data from Unity Catalog: {e}")
    raise

# Transformation logic
try:
    # Join hospital statistics with sales assignments
    hospital_sales_df = hospital_stats_df.join(
        sales_assignments_df,
        on=["Hospital_ID", "Hospital_Name"],
        how="inner"
    )

    # Join employment details with compensation guidelines
    employment_compensation_df = employment_details_df.join(
        compensation_guidelines_df,
        on="Associate_ID",
        how="inner"
    )

    # Calculate total compensation
    employment_compensation_df = employment_compensation_df.withColumn(
        "Compensation",
        F.col("Base_Salary") + (F.col("Commission_Percentage") / 100 * F.col("Base_Salary")) + F.col("Bonus")
    )

    # Select relevant fields
    selected_employment_df = employment_compensation_df.select(
        "Associate_ID", "Associate_Name", "Compensation", "Region", "Employment_Type", "Years_of_Experience", "Department"
    )

    # Join logistics channels with growth opportunities
    logistics_growth_df = logistics_channels_df.join(
        growth_opportunities_df,
        on=["Channel_ID", "Channel_Type", "Hospital_ID"],
        how="inner"
    )

    # Join compensation data with channel growth data
    compensation_growth_df = selected_employment_df.join(
        logistics_growth_df,
        on="Hospital_ID",
        how="inner"
    )

    # Select fields for further processing
    selected_growth_df = compensation_growth_df.select(
        "Hospital_ID", "Channel_Type", "Growth_Opportunities", "Projected_Growth_Rate"
    )

    # Join selected data with company goals
    growth_goals_df = selected_growth_df.join(
        company_goals_df,
        on=["Hospital_ID", "Channel_Type"],
        how="inner"
    )

    # Ensure unique records
    unique_growth_goals_df = growth_goals_df.dropDuplicates(["Hospital_ID", "Channel_Type", "Projected_Growth_Rate", "Investment_Planned"])

    # Join historical sales with third-party sales trends
    sales_trends_df = historical_sales_df.join(
        third_party_sales_trends_df,
        on="Channel_Type",
        how="inner"
    )

    # Ensure unique sales records
    unique_sales_trends_df = sales_trends_df.dropDuplicates(["Year", "Channel_Type", "Sales_Revenue"])

    # Combine unique sales data with growth data
    combined_sales_growth_df = unique_sales_trends_df.join(
        unique_growth_goals_df,
        on=["Hospital_ID", "Channel_ID", "Channel_Type"],
        how="inner"
    )

    # Generate rows for target years
    target_years_df = combined_sales_growth_df.withColumn(
        "Target Year",
        F.expr("sequence(2023, 2026)")
    ).selectExpr("explode(`Target Year`) as `Target Year`", "*")

    # Calculate projected sales growth rate
    target_years_df = target_years_df.withColumn(
        "projected_sales_growth_rate",
        F.when(F.col("Target Year") == 2024, F.col("Projected_Growth_Rate") + (F.col("Projected_Growth_Rate") / 100))
        .when(F.col("Target Year") == 2025, (F.col("Projected_Growth_Rate") + (F.col("Projected_Growth_Rate") / 100)) + (F.col("Projected_Growth_Rate") / 100))
        .when(F.col("Target Year") == 2026, ((F.col("Projected_Growth_Rate") + (F.col("Projected_Growth_Rate") / 100)) + (F.col("Projected_Growth_Rate") / 100)) + (F.col("Projected_Growth_Rate") / 100))
        .otherwise(F.col("Projected_Growth_Rate"))
    )

    # Calculate projected investments
    target_years_df = target_years_df.withColumn(
        "projected_investments",
        F.when(F.col("Target Year") == 2024, F.col("Investment_Planned") * (F.col("projected_sales_growth_rate") / 100))
        .when(F.col("Target Year") == 2025, F.col("Investment_Planned") * (1 + F.col("projected_sales_growth_rate") / 100))
        .when(F.col("Target Year") == 2026, F.col("Investment_Planned") * (1 + F.col("projected_sales_growth_rate") / 100))
        .otherwise(F.col("Investment_Planned"))
    )

    # Calculate projected revenue
    target_years_df = target_years_df.withColumn(
        "Projected Revenue",
        F.when(F.col("Target Year") == 2024, F.col("Sales_Revenue") * (F.col("projected_sales_growth_rate") / 100))
        .when(F.col("Target Year") == 2025, F.col("Sales_Revenue") * (1 + F.col("projected_sales_growth_rate") / 100))
        .when(F.col("Target Year") == 2026, F.col("Sales_Revenue") * (1 + F.col("projected_sales_growth_rate") / 100))
        .otherwise(F.col("Sales_Revenue"))
    )

    # Filter records based on target year
    filtered_df = target_years_df.filter(F.col("Target Year") > 2023)

    # Select fields for final output
    final_output_df = filtered_df.select(
        "Channel_Type", "Hospital_ID", "Market_Trend", "Political_Impact", "Economic_Impact", "Target Year",
        "projected_sales_growth_rate", "projected_investments", "Projected Revenue"
    )

    # Sort records by target year
    sorted_output_df = final_output_df.orderBy("Target Year")

except Exception as e:
    logger.error(f"Error during transformation: {e}")
    raise

# Output handling
try:
    # Drop existing table if necessary
    spark.sql("DROP TABLE IF EXISTS catalog.db.sales_prediction_output")

    # Write the processed data to Unity Catalog table using Delta format
    sorted_output_df.write.format("delta").mode("overwrite").saveAsTable("catalog.db.sales_prediction_output")
except Exception as e:
    logger.error(f"Error writing data to Unity Catalog: {e}")
    raise

logger.info("ETL process completed successfully.")
